In [19]:
!pip install -U transformers --quiet

Model page: https://huggingface.co/google-bert/bert-base-uncased

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google-bert/bert-base-uncased)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [20]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = "false" 

In [21]:
!pip install beautifulsoup4 

In [22]:
import torch

In [23]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from transformers import AutoTokenizer , AutoModelForSequenceClassification

In [24]:
import requests

In [25]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [26]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="google-bert/bert-base-uncased") # This doesn't distinguish between upper and lower cases.
# Debugging Step : pipe("The capital of France is [MASK]")

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [27]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [28]:
tokens = tokenizer.encode('This is a grest review of the current market trend!!',return_tensors='pt') # 'pt' stands for 'pytorch'
print(tokens[0])
tokenizer.decode(tokens[0])

tensor([  101, 10372, 10127,   143, 50565, 10607, 12893, 10108, 10103, 14385,
        15200, 23853,   106,   106,   102])


'[CLS] this is a grest review of the current market trend!! [SEP]'

In [29]:
result = model(tokens)
print(result)
torch.argmax(result.logits)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.1857, -1.1264, -0.0310,  0.9309,  1.0920]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


tensor(4)

In [47]:
r = requests.get('https://www.zomato.com/kanpur/punjab-grill-mall-road/reviews')
soup = BeautifulSoup (r.text , 'html.parser')
regex = re.compile('.*Punjab Grill Reviews.*')
results = soup.find_all('p' , {'Class':regex})
reviews = [result.text for result in results]

In [32]:
df=pd.DataFrame(np.array(reviews) , columns = ['reviews'])
df.head()
# print(df.shape[0])

,reviews


In [33]:
def sentiment_analysis(review):
    tokens = tokenizer.encode(review , return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
df['sentiment_score'] = df['reviews'].apply(lambda x:sentiment_analysis(x[:512]))

In [ ]:
df['sentiment_score'].value_counts()